# Notebook 5.2: Baichuan-13B

## Overview

## System Recommended Requirement

## Inference

### Create Prompt Template

In [1]:
BAICHUAN_PROMPT_FORMAT = "<human>{prompt} <bot>"

### Load Model and Tokenizer

In [10]:
from bigdl.llm.transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

model_path = "baichuan-inc/Baichuan-13B-Chat"
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_4bit=True,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:12<00:00,  4.02s/it]


### Save and Reload

### Predict

In [11]:
import time
import torch

prompt = "AI是什么？"
n_predict = 128
with torch.inference_mode():
        prompt = BAICHUAN_PROMPT_FORMAT.format(prompt=prompt)
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        st = time.time()
        # if your selected model is capable of utilizing previous key/value attentions
        # to enhance decoding speed, but has `"use_cache": false` in its model config,
        # it is important to set `use_cache=True` explicitly in the `generate` function
        # to obtain optimal performance with BigDL-LLM INT4 optimizations
        output = model.generate(input_ids,
                                max_new_tokens=n_predict)
        end = time.time()
        output_str = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f'Inference time: {end-st} s')
        print('-'*20, 'Prompt', '-'*20)
        print(prompt)
        print('-'*20, 'Output', '-'*20)
        print(output_str)

Inference time: 9.203657627105713 s
-------------------- Prompt --------------------
<human>AI是什么？ <bot>
-------------------- Output --------------------
<human>AI是什么？ <bot>人工智能(Artificial Intelligence，简称AI)是指由人制造出来的系统所表现出的智能，通常是通过计算机程序和传感器实现的。这种智能可以模拟人类的思维、学习、感知、语言理解等能力，也可以完成一些需要人类智慧才能解决的问题。
